## Import Libraries

In [45]:
import pandas as pd
import numpy as np
import glob
import datetime

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Helper Functions

In [3]:
def filter_dataframes(path, min_date):
    '''
    Return the concated dataframe for all the files in a folder with some filters over date and columns
    
    path: folder path (str)
    min_date: minimum date (str YYYY-MM-DD)
    '''
    
    appended_data = []
    
    for filename in glob.glob(f'{path}*.csv'):
        
        file = filename.split('\\')[1]
        print('reading:', file)
        
        df = pd.read_csv(path + file)
        df['Data'] = pd.to_datetime(df['Data'])
        
        min_date = pd.to_datetime(min_date)
        
        df = df[df['Data'] >= min_date ].drop(columns=['index', 'state', 'station', 'station_code', 'latitude', 'longitude', 'height'])
        
        appended_data.append(df) 
        
    return pd.concat(appended_data)

## Readind Weather Data

In [4]:
df_weather = filter_dataframes('original_bases/', '2017-01-01')

reading: central_west.csv
reading: north.csv
reading: northeast.csv
reading: south.csv
reading: southeast.csv


In [5]:
# Treating invalid information for columns

for col in df_weather.drop(columns=['Data', 'Hora', 'region']).columns:
    
    print(f'treating data for {col}', end=' ')
    min_value = df_weather[col].min()
    
    df_weather = df_weather[df_weather[col]!=min_value]
    
    print(f'New min value = {df_weather[col].min()}')

treating data for PRECIPITAÇÃO TOTAL, HORÁRIO (mm) New min value = 0.0
treating data for PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB) New min value = 720.1
treating data for PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB) New min value = 720.3
treating data for PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB) New min value = 720.0
treating data for RADIACAO GLOBAL (Kj/m²) New min value = 0
treating data for TEMPERATURA DO AR - BULBO SECO, HORARIA (°C) New min value = -9.0
treating data for TEMPERATURA DO PONTO DE ORVALHO (°C) New min value = -42.5
treating data for TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C) New min value = -8.2
treating data for TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C) New min value = -8.6
treating data for TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C) New min value = -35.0
treating data for TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C) New min value = -35.0
treating data for UMIDADE REL. MAX. NA HORA ANT. (AUT) (%) New min value = 3
treating data for 

In [6]:
# Fixing Dtype for Hour column

df_weather['Hora'] = pd.to_datetime(df_weather['Hora']).dt.hour

In [7]:
df_left = df_weather.drop(columns=['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)']).groupby(['Data', 'Hora']).mean(numeric_only=True).reset_index()
df_right = df_weather.groupby(['Data', 'Hora'])[['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)']].sum(numeric_only=True).reset_index()

df_weather = pd.merge(df_left, df_right, how='inner', on=['Data', 'Hora'])
del df_left, df_right

### Read Population Data

In [68]:
df_pop = pd.read_csv('populacao_municipio.csv')
df_pop = df_pop.groupby("ano")[["populacao"]].sum().reset_index()

In [69]:
df_pop["last_year"] = df_pop["ano"]-1
df_pop = df_pop.merge(df_pop, how='inner',left_on='last_year',right_on='ano',suffixes=('', '_passado'))
df_pop = df_pop.drop(columns=["last_year","ano_passado","last_year_passado"])
df_pop = df_pop[df_pop["ano"]>=2017]
df_pop["variacao"]=df_pop["populacao"]-df_pop["populacao_passado"]

In [70]:
df_data = df_weather[["Data"]].drop_duplicates()
df_data["year"] = df_data["Data"].dt.year

In [71]:
df_pop = df_pop.merge(df_data, how='inner',left_on='ano',right_on='year',suffixes=('', ''))
df_pop = df_pop.drop(columns=["year"])

In [73]:
df_pop["day_of_year"]=df_pop["Data"].dt.day_of_year
df_pop["populacao_dia"]=df_pop["populacao_passado"] + df_pop["day_of_year"]*df_pop["variacao"]/365

,ano,populacao,populacao_passado,variacao,Data,day_of_year,populacao_dia
0,2017,207660929.0,206081432.0,1579497.0,2017-01-01,1,2.060858e+08
1,2017,207660929.0,206081432.0,1579497.0,2017-01-02,2,2.060901e+08
2,2017,207660929.0,206081432.0,1579497.0,2017-01-03,3,2.060944e+08
3,2017,207660929.0,206081432.0,1579497.0,2017-01-04,4,2.060987e+08
4,2017,207660929.0,206081432.0,1579497.0,2017-01-05,5,2.061031e+08
...,...,...,...,...,...,...,...
1576,2021,213317639.0,211755692.0,1561947.0,2021-04-26,116,2.122521e+08
1577,2021,213317639.0,211755692.0,1561947.0,2021-04-27,117,2.122564e+08
1578,2021,213317639.0,211755692.0,1561947.0,2021-04-28,118,2.122607e+08
1579,2021,213317639.0,211755692.0,1561947.0,2021-04-29,119,2.122649e+08


### Acquiring target information

In [11]:
df_target = pd.read_csv('energy_demand_hourly_brazil.csv')

In [12]:
df_target['Data'] = pd.to_datetime(df_target['index'])

In [13]:
df_target['index'] = df_target['index'].apply(lambda x: x.split()[0])

In [14]:
df_target['Hora'] = df_target['Data'].dt.hour

In [15]:
df_target['Data'] = pd.to_datetime(df_target['index'])

In [16]:
df_target.drop(columns=['index'], inplace=True)

In [17]:
df_target

,hourly_demand,Data,Hora
0,34673.900,2000-01-01,0
1,33503.000,2000-01-01,1
2,32287.600,2000-01-01,2
3,31059.400,2000-01-01,3
4,30272.700,2000-01-01,4
...,...,...,...
201313,75825.344,2022-12-31,20
201314,69894.955,2022-12-31,21
201315,64612.355,2022-12-31,22
201316,61215.376,2022-12-31,23


## Joining bases

In [18]:
df_final = pd.merge(df_target, df_weather, how='inner', on=['Data', 'Hora'])

In [74]:
df_final = pd.merge(df_final, df_pop[['Data','populacao_dia']], how='inner', on=['Data'])

In [75]:
df_final.to_csv('df_final.csv', index=False)